In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
name_counts = charity_df.NAME.value_counts()
name_counts.head(25)

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [4]:
# Check the unique value counts to see if binning is required
classification_counts = charity_df.CLASSIFICATION.value_counts()
classification_counts.head(25)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
Name: CLASSIFICATION, dtype: int64

In [5]:
# Check the unique value counts to see if binning is required
affiliation_counts = charity_df.AFFILIATION.value_counts()
affiliation_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [6]:
#Dropping Names Col.
charity_df = charity_df.drop("NAME",1)

# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

# Question
What variable(s) are considered the target for your model?

*IS_SUCCESSFUL

What variable(s) are considered to be the features for your model?

*All Others will need to group Classifications:

C1000    17326

C2000     6074

C1200     4837

C3000     1918

C2100     1883

C7000      777

C1700      287

C4000      194

C5000      116

C1270      114

C2700      104

Everything else will be other

What variable(s) are neither and should be removed from the input data?

*Names column can be dropped

In [7]:
# Determine which values to replace
replace_classifications = list(classification_counts[classification_counts < 100].index)

# Replace in DataFrame
for CLASSIFICATION in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(CLASSIFICATION,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [8]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df.CLASSIFICATION.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['CLASSIFICATION'])
encode_df.head()

,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1270,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2700,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df2 = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the dataframe
encode_df2.columns = enc.get_feature_names(charity_cat)
encode_df2.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge the two DataFrames together and drop the Country column
charity_df = charity_df.merge(encode_df,left_index=True,right_index=True).drop("CLASSIFICATION",1)
charity_df

,EIN,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,CLASSIFICATION_C1270,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2700,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,10520599,T10,Independent,ProductDev,Association,1,0,N,5000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10531628,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10547893,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,10553066,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10556103,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,T4,Independent,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,996010315,T4,CompanySponsored,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34296,996012607,T3,CompanySponsored,Preservation,Association,1,0,N,5000,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,996015768,T5,Independent,ProductDev,Association,1,0,N,5000,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# iterating the columns 
for col in encode_df2.columns: 
    print(col) 

APPLICATION_TYPE_T10
APPLICATION_TYPE_T12
APPLICATION_TYPE_T13
APPLICATION_TYPE_T14
APPLICATION_TYPE_T15
APPLICATION_TYPE_T17
APPLICATION_TYPE_T19
APPLICATION_TYPE_T2
APPLICATION_TYPE_T25
APPLICATION_TYPE_T29
APPLICATION_TYPE_T3
APPLICATION_TYPE_T4
APPLICATION_TYPE_T5
APPLICATION_TYPE_T6
APPLICATION_TYPE_T7
APPLICATION_TYPE_T8
APPLICATION_TYPE_T9
AFFILIATION_CompanySponsored
AFFILIATION_Family/Parent
AFFILIATION_Independent
AFFILIATION_National
AFFILIATION_Other
AFFILIATION_Regional
CLASSIFICATION_C1000
CLASSIFICATION_C1200
CLASSIFICATION_C1270
CLASSIFICATION_C1700
CLASSIFICATION_C2000
CLASSIFICATION_C2100
CLASSIFICATION_C2700
CLASSIFICATION_C3000
CLASSIFICATION_C4000
CLASSIFICATION_C5000
CLASSIFICATION_C7000
CLASSIFICATION_Other
USE_CASE_CommunityServ
USE_CASE_Heathcare
USE_CASE_Other
USE_CASE_Preservation
USE_CASE_ProductDev
ORGANIZATION_Association
ORGANIZATION_Co-operative
ORGANIZATION_Corporation
ORGANIZATION_Trust
INCOME_AMT_0
INCOME_AMT_1-9999
INCOME_AMT_10000-24999
INCOME_AMT_1

In [12]:
charity_df = charity_df.drop("APPLICATION_TYPE",1)
charity_df = charity_df.drop("AFFILIATION",1)
charity_df = charity_df.drop("USE_CASE",1)
charity_df = charity_df.drop("ORGANIZATION",1)
charity_df = charity_df.drop("INCOME_AMT",1)
charity_df = charity_df.drop("SPECIAL_CONSIDERATIONS",1)

In [13]:
# Merge the two DataFrames together and drop the Country column
charity_df = charity_df.merge(encode_df2,left_index=True,right_index=True)
charity_df

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000_x,CLASSIFICATION_C1200_x,CLASSIFICATION_C1270_x,CLASSIFICATION_C1700_x,CLASSIFICATION_C2000_x,CLASSIFICATION_C2100_x,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,996010315,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,996012607,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,996015768,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# iterating the columns 
for col in charity_df.columns: 
    print(col) 

EIN
STATUS
ASK_AMT
IS_SUCCESSFUL
CLASSIFICATION_C1000_x
CLASSIFICATION_C1200_x
CLASSIFICATION_C1270_x
CLASSIFICATION_C1700_x
CLASSIFICATION_C2000_x
CLASSIFICATION_C2100_x
CLASSIFICATION_C2700_x
CLASSIFICATION_C3000_x
CLASSIFICATION_C4000_x
CLASSIFICATION_C5000_x
CLASSIFICATION_C7000_x
CLASSIFICATION_Other_x
APPLICATION_TYPE_T10
APPLICATION_TYPE_T12
APPLICATION_TYPE_T13
APPLICATION_TYPE_T14
APPLICATION_TYPE_T15
APPLICATION_TYPE_T17
APPLICATION_TYPE_T19
APPLICATION_TYPE_T2
APPLICATION_TYPE_T25
APPLICATION_TYPE_T29
APPLICATION_TYPE_T3
APPLICATION_TYPE_T4
APPLICATION_TYPE_T5
APPLICATION_TYPE_T6
APPLICATION_TYPE_T7
APPLICATION_TYPE_T8
APPLICATION_TYPE_T9
AFFILIATION_CompanySponsored
AFFILIATION_Family/Parent
AFFILIATION_Independent
AFFILIATION_National
AFFILIATION_Other
AFFILIATION_Regional
CLASSIFICATION_C1000_y
CLASSIFICATION_C1200_y
CLASSIFICATION_C1270_y
CLASSIFICATION_C1700_y
CLASSIFICATION_C2000_y
CLASSIFICATION_C2100_y
CLASSIFICATION_C2700_y
CLASSIFICATION_C3000_y
CLASSIFICATION_C400

In [15]:
# Split our preprocessed data into our features and target arrays
y = charity_df["SPECIAL_CONSIDERATIONS_Y"].values
X = charity_df.drop(["SPECIAL_CONSIDERATIONS_Y","SPECIAL_CONSIDERATIONS_N"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  14
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 14)                980       
_________________________________________________________________
dense_22 (Dense)             (None, 12)                180       
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 13        
Total params: 1,173
Trainable params: 1,173
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 1s 57us/sample - loss: 7870.7641 - accuracy: 0.9966
Epoch 2/100
25724/25724 [==============================] - 1s 28us/sample - loss: 10929.2031 - accuracy: 0.9980
Epoch 3/100
25724/25724 [==============================] - 1s 38us/sample - loss: 10984.1718 - accuracy: 0.9981
Epoch 4/100
25724/25724 [==============================] - 1s 34us/sample - loss: 12598.6674 - accuracy: 0.9981
Epoch 5/100
25724/25724 [==============================] - 1s 34us/sample - loss: 14856.1477 - accuracy: 0.9981
Epoch 6/100
25724/25724 [==============================] - 1s 29us/sample - loss: 11498.9289 - accuracy: 0.9982
Epoch 7/100
25724/25724 [==============================] - 1s 28us/sample - loss: 9507.1188 - accuracy: 0.9980
Epoch 8/100
25724/25724 [==============================] - 1s 29us/sample - loss: 13123.9191 - accuracy: 0.9980
Epoch 9/100
25724/25724 [==============================] - 1s 29us/sample - loss: 1

25724/25724 [==============================] - 1s 30us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 75/100
25724/25724 [==============================] - 1s 28us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 76/100
25724/25724 [==============================] - 1s 29us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 77/100
25724/25724 [==============================] - 1s 29us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 78/100
25724/25724 [==============================] - 1s 29us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 79/100
25724/25724 [==============================] - 1s 29us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 80/100
25724/25724 [==============================] - 1s 29us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 81/100
25724/25724 [==============================] - 1s 27us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 82/100
25724/25724 [==============================] - 1s 30us/sample - loss: 0.0063 - accuracy: 0.9992
Epoch 83/100
25724/25724 [======

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.0066 - accuracy: 0.9992
Loss: 0.00661950351419226, Accuracy: 0.9991836547851562


# Adding Checkpoints

In [21]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
   32/25724 [..............................] - ETA: 6:10 - loss: 6.1971e-04 - accuracy: 1.0000
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1088/25724 [>.............................] - ETA: 11s - loss: 0.0071 - accuracy: 0.9991     
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2208/25724 [=>............................] - ETA: 5s - loss: 0.0144 - accuracy: 0.9982 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3552/25724 [===>..........................] - ETA: 3s - loss: 0.0112 - accuracy: 0.9986
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4992/25724 [====>.........................] - ETA: 2s - loss: 0.0095 - accuracy: 0.9988
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6368/25724 [======>.......................] - ETA: 2s - loss: 0.0076 - accuracy: 0.9991
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 7616/25724 [=====

12672/25724 [=============>................] - ETA: 0s - loss: 0.0078 - accuracy: 0.9991
Epoch 00003: saving model to checkpoints/weights.03.hdf5

Epoch 00003: saving model to checkpoints/weights.03.hdf5
14080/25724 [===============>..............] - ETA: 0s - loss: 0.0071 - accuracy: 0.9992
Epoch 00003: saving model to checkpoints/weights.03.hdf5
15584/25724 [=================>............] - ETA: 0s - loss: 0.0064 - accuracy: 0.9993
Epoch 00003: saving model to checkpoints/weights.03.hdf5
17120/25724 [==================>...........] - ETA: 0s - loss: 0.0063 - accuracy: 0.9993
Epoch 00003: saving model to checkpoints/weights.03.hdf5

Epoch 00003: saving model to checkpoints/weights.03.hdf5
18368/25724 [====================>.........] - ETA: 0s - loss: 0.0067 - accuracy: 0.9992
Epoch 00003: saving model to checkpoints/weights.03.hdf5
20032/25724 [======================>.......] - ETA: 0s - loss: 0.0065 - accuracy: 0.9993
Epoch 00003: saving model to checkpoints/weights.03.hdf5

Epoch 0

   32/25724 [..............................] - ETA: 1s - loss: 5.1899e-04 - accuracy: 1.0000
Epoch 00006: saving model to checkpoints/weights.06.hdf5
 1152/25724 [>.............................] - ETA: 1s - loss: 4.1923e-04 - accuracy: 1.0000
Epoch 00006: saving model to checkpoints/weights.06.hdf5
 2144/25724 [=>............................] - ETA: 1s - loss: 4.2608e-04 - accuracy: 1.0000
Epoch 00006: saving model to checkpoints/weights.06.hdf5

Epoch 00006: saving model to checkpoints/weights.06.hdf5
 3456/25724 [===>..........................] - ETA: 0s - loss: 4.2206e-04 - accuracy: 1.0000
Epoch 00006: saving model to checkpoints/weights.06.hdf5
 5088/25724 [====>.........................] - ETA: 0s - loss: 0.0062 - accuracy: 0.9994    
Epoch 00006: saving model to checkpoints/weights.06.hdf5

Epoch 00006: saving model to checkpoints/weights.06.hdf5
 7200/25724 [=======>......................] - ETA: 0s - loss: 0.0045 - accuracy: 0.9996
Epoch 00006: saving model to checkpoints/weig

Epoch 00008: saving model to checkpoints/weights.08.hdf5
14560/25724 [===============>..............] - ETA: 0s - loss: 0.0060 - accuracy: 0.9992
Epoch 00008: saving model to checkpoints/weights.08.hdf5
15680/25724 [=================>............] - ETA: 0s - loss: 0.0057 - accuracy: 0.9992
Epoch 00008: saving model to checkpoints/weights.08.hdf5
16832/25724 [==================>...........] - ETA: 0s - loss: 0.0053 - accuracy: 0.9993
Epoch 00008: saving model to checkpoints/weights.08.hdf5
18048/25724 [====================>.........] - ETA: 0s - loss: 0.0057 - accuracy: 0.9993
Epoch 00008: saving model to checkpoints/weights.08.hdf5
19456/25724 [=====================>........] - ETA: 0s - loss: 0.0056 - accuracy: 0.9993
Epoch 00008: saving model to checkpoints/weights.08.hdf5

Epoch 00008: saving model to checkpoints/weights.08.hdf5
20768/25724 [=======================>......] - ETA: 0s - loss: 0.0056 - accuracy: 0.9993
Epoch 00008: saving model to checkpoints/weights.08.hdf5
22304/257

 1440/25724 [>.............................] - ETA: 0s - loss: 0.0097 - accuracy: 0.9986    
Epoch 00011: saving model to checkpoints/weights.11.hdf5

Epoch 00011: saving model to checkpoints/weights.11.hdf5
 2816/25724 [==>...........................] - ETA: 0s - loss: 0.0099 - accuracy: 0.9986
Epoch 00011: saving model to checkpoints/weights.11.hdf5
 4320/25724 [====>.........................] - ETA: 0s - loss: 0.0112 - accuracy: 0.9984
Epoch 00011: saving model to checkpoints/weights.11.hdf5
 5632/25724 [=====>........................] - ETA: 0s - loss: 0.0100 - accuracy: 0.9986
Epoch 00011: saving model to checkpoints/weights.11.hdf5

Epoch 00011: saving model to checkpoints/weights.11.hdf5
 6976/25724 [=======>......................] - ETA: 0s - loss: 0.0100 - accuracy: 0.9986
Epoch 00011: saving model to checkpoints/weights.11.hdf5
 8544/25724 [========>.....................] - ETA: 0s - loss: 0.0083 - accuracy: 0.9988
Epoch 00011: saving model to checkpoints/weights.11.hdf5

Epo

...

 6560/25724 [======>.......................] - ETA: 0s - loss: 0.0061 - accuracy: 0.9991
Epoch 00090: saving model to checkpoints/weights.90.hdf5
 7968/25724 [========>.....................] - ETA: 0s - loss: 0.0059 - accuracy: 0.9991
Epoch 00090: saving model to checkpoints/weights.90.hdf5

Epoch 00090: saving model to checkpoints/weights.90.hdf5
 9152/25724 [=========>....................] - ETA: 0s - loss: 0.0058 - accuracy: 0.9991
Epoch 00090: saving model to checkpoints/weights.90.hdf5
10592/25724 [===========>..................] - ETA: 0s - loss: 0.0052 - accuracy: 0.9992
Epoch 00090: saving model to checkpoints/weights.90.hdf5
12128/25724 [=============>................] - ETA: 0s - loss: 0.0060 - accuracy: 0.9991
Epoch 00090: saving model to checkpoints/weights.90.hdf5

Epoch 00090: saving model to checkpoints/weights.90.hdf5
13408/25724 [==============>...............] - ETA: 0s - loss: 0.0055 - accuracy: 0.9992
Epoch 00090: saving model to checkpoints/weights.90.hdf5
14976/25

20128/25724 [======================>.......] - ETA: 0s - loss: 0.0060 - accuracy: 0.9991
Epoch 00092: saving model to checkpoints/weights.92.hdf5
21536/25724 [========================>.....] - ETA: 0s - loss: 0.0057 - accuracy: 0.9992
Epoch 00092: saving model to checkpoints/weights.92.hdf5
22592/25724 [=========================>....] - ETA: 0s - loss: 0.0054 - accuracy: 0.9992
Epoch 00092: saving model to checkpoints/weights.92.hdf5
23840/25724 [==========================>...] - ETA: 0s - loss: 0.0052 - accuracy: 0.9992
Epoch 00092: saving model to checkpoints/weights.92.hdf5

Epoch 00092: saving model to checkpoints/weights.92.hdf5
25724/25724 [==============================] - 1s 37us/sample - loss: 0.0053 - accuracy: 0.9992
Epoch 93/100
   32/25724 [..............................] - ETA: 1s - loss: 7.9035e-04 - accuracy: 1.0000
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 1120/25724 [>.............................] - ETA: 1s - loss: 0.0060 - accuracy: 0.9991    
Epoch 

 5408/25724 [=====>........................] - ETA: 0s - loss: 0.0029 - accuracy: 0.9996
Epoch 00095: saving model to checkpoints/weights.95.hdf5
 6720/25724 [======>.......................] - ETA: 0s - loss: 0.0033 - accuracy: 0.9996
Epoch 00095: saving model to checkpoints/weights.95.hdf5
 7712/25724 [=======>......................] - ETA: 0s - loss: 0.0045 - accuracy: 0.9994
Epoch 00095: saving model to checkpoints/weights.95.hdf5
 8832/25724 [=========>....................] - ETA: 0s - loss: 0.0047 - accuracy: 0.9993
Epoch 00095: saving model to checkpoints/weights.95.hdf5
 9920/25724 [==========>...................] - ETA: 0s - loss: 0.0043 - accuracy: 0.9994
Epoch 00095: saving model to checkpoints/weights.95.hdf5
11488/25724 [============>.................] - ETA: 0s - loss: 0.0038 - accuracy: 0.9995
Epoch 00095: saving model to checkpoints/weights.95.hdf5

Epoch 00095: saving model to checkpoints/weights.95.hdf5
12928/25724 [==============>...............] - ETA: 0s - loss: 0.0

18432/25724 [====================>.........] - ETA: 0s - loss: 0.0050 - accuracy: 0.9993
Epoch 00097: saving model to checkpoints/weights.97.hdf5
19744/25724 [======================>.......] - ETA: 0s - loss: 0.0053 - accuracy: 0.9992
Epoch 00097: saving model to checkpoints/weights.97.hdf5
21216/25724 [=======================>......] - ETA: 0s - loss: 0.0056 - accuracy: 0.9992
Epoch 00097: saving model to checkpoints/weights.97.hdf5

Epoch 00097: saving model to checkpoints/weights.97.hdf5
22528/25724 [=========================>....] - ETA: 0s - loss: 0.0056 - accuracy: 0.9992
Epoch 00097: saving model to checkpoints/weights.97.hdf5
24032/25724 [===========================>..] - ETA: 0s - loss: 0.0056 - accuracy: 0.9992
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25472/25724 [============================>.] - ETA: 0s - loss: 0.0055 - accuracy: 0.9992
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25724/25724 [==============================] - 1s 39us/sample - lo

Epoch 00100: saving model to checkpoints/weights.100.hdf5
 4384/25724 [====>.........................] - ETA: 0s - loss: 0.0086 - accuracy: 0.9986
Epoch 00100: saving model to checkpoints/weights.100.hdf5

Epoch 00100: saving model to checkpoints/weights.100.hdf5
 5760/25724 [=====>........................] - ETA: 0s - loss: 0.0068 - accuracy: 0.9990
Epoch 00100: saving model to checkpoints/weights.100.hdf5
 7232/25724 [=======>......................] - ETA: 0s - loss: 0.0063 - accuracy: 0.9990
Epoch 00100: saving model to checkpoints/weights.100.hdf5
 8448/25724 [========>.....................] - ETA: 0s - loss: 0.0055 - accuracy: 0.9992
Epoch 00100: saving model to checkpoints/weights.100.hdf5
 9248/25724 [=========>....................] - ETA: 0s - loss: 0.0057 - accuracy: 0.9991
Epoch 00100: saving model to checkpoints/weights.100.hdf5
10336/25724 [===========>..................] - ETA: 0s - loss: 0.0052 - accuracy: 0.9992
Epoch 00100: saving model to checkpoints/weights.100.hdf5
1

In [25]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  4
hidden_nodes_layer2 = 2

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.0076 - accuracy: 0.9992
Loss: 0.007633456578033599, Accuracy: 0.9991836547851562


In [31]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
#hidden_nodes_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 8)                 560       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 9         
Total params: 569
Trainable params: 569
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 1s 46us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 2/100
25724/25724 [==============================] - 1s 30us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 3/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 4/100
25724/25724 [==============================] - 1s 33us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 5/100
25724/25724 [==============================] - 1s 32us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 6/100
25724/25724 [==============================] - 1s 32us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 7/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 8/100
25724/25724 [==============================] - 1s 35us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 9/100
25724/25724 [==============================] - 1s 30us/sample

25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 73/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 74/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 75/100
25724/25724 [==============================] - 1s 28us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 76/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 77/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 78/100
25724/25724 [==============================] - 1s 30us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 79/100
25724/25724 [==============================] - 1s 28us/sample - loss: 15.2374 - accuracy: 7.7748e-04
Epoch 80/100
25724/25724 [==============================] - 1s 29us/sample - loss: 15.2374 - accuracy

# Saving Model & Loading it

In [33]:
# Export our model to HDF5 file
nn_new.save("trained_alphabet.h5")

In [34]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_alphabet.h5')

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.0076 - accuracy: 0.9992
Loss: 0.007633456578033599, Accuracy: 0.9991836547851562
